# pyodbc連接資料庫
    # 安裝：conda install -c anaconda pyodbc

In [7]:
import pyodbc
server = '127.0.0.1, 1433'
database = 'dbDemo' 
username = 'sa' 
password = 'as' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + 
                      server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
cursor = cnxn.cursor()
# 1.Sample select query
cursor.execute("select * from tProduct") 
rows = cursor.fetchall()
for row in rows:
    print(row)
    print('---')
# 2.Sample insert
try:
    cnxn.autocommit = False
    params = [ ('A', 10, 5), ('B', 8, 6) ]
    cursor.executemany("insert into tProduct(fName, fPrice, fCost) values (?, ?, ?)", params)
except pyodbc.DatabaseError as err:
    cnxn.rollback()
else:
    cnxn.commit()
finally:
    cnxn.autocommit = True

print('---------再次查詢---------')
cursor.execute("select * from tProduct") 
rows = cursor.fetchall()
for row in rows:
    print(row)
    print('---')

(1, '防疫口罩大人用', Decimal('7.0000'), Decimal('2.0000'), 300)
---
(2, '防疫口罩小孩用', Decimal('5.0000'), Decimal('2.0000'), 600)
---
(3, '臺酒酒精', Decimal('30.0000'), Decimal('15.0000'), 1000)
---
---------再次查詢---------
(1, '防疫口罩大人用', Decimal('7.0000'), Decimal('2.0000'), 300)
---
(2, '防疫口罩小孩用', Decimal('5.0000'), Decimal('2.0000'), 600)
---
(3, '臺酒酒精', Decimal('30.0000'), Decimal('15.0000'), 1000)
---
(8, 'A', Decimal('10.0000'), Decimal('5.0000'), None)
---
(9, 'B', Decimal('8.0000'), Decimal('6.0000'), None)
---


# 爬取資料主程式


In [13]:
def crawal():
    import datetime
    today = datetime.datetime.now()
    print("開始爬蟲，時間：" + today.strftime("%Y%m%d"))
    
    year_s = str(today.year - 1911)
    month_s = str(today.month)

    year = "{:0>3d}".format(today.year - 1911)
    month = today.strftime("%m")
    day = today.strftime("%d")
    
    try:
        from selenium import webdriver
        from selenium.webdriver.support.ui import Select

        # 1.執行程式打開新的瀏覽器頁面
        browser = webdriver.Chrome(executable_path='chromedriver')
        browser.get("https://amis.afa.gov.tw/m_fruit/FruitProdDayTransInfo.aspx")
            # 選時間
        browser.find_element_by_id('ctl00_contentPlaceHolder_ucDateScope_rblDateScope_0').click()
        browser.find_element_by_id('ctl00_contentPlaceHolder_ucSolarLunar_radlSolarLunar_0').click()
        browser.find_element_by_id('ctl00_contentPlaceHolder_txtSTransDate').click()
        selectSite = Select(browser.find_element_by_id("sYear_amis"))
        selectSite.select_by_value(year_s)
        selectYear = Select(browser.find_element_by_id("sMonth_amis"))
        selectYear.select_by_value(month_s)
        onclick_str = "setValue('" + year + "/" + month + "/" + day + "');"
        browser.find_element_by_xpath('//td[@onclick="' + onclick_str + '"]').click()
            # 選市場
        browser.find_element_by_id('ctl00_contentPlaceHolder_ucFruitMarket_radlMarketRange_0').click()
            # 選產品
        browser.find_element_by_id('ctl00_contentPlaceHolder_ucFruitProduct_radlProductRange_0').click()
            # 查詢
        browser.find_element_by_id('ctl00_contentPlaceHolder_btnQuery').click()

        # 2.取得資料（等瀏覽器資料出現後才執行）
        import time
        time.sleep(10)
        html_source = browser.page_source

            # 關閉瀏覽器
        browser.quit()
        from bs4 import BeautifulSoup
        soup = BeautifulSoup(html_source, 'html.parser')
        # print(soup)
        html = soup.find("div", attrs={'id': 'ctl00_contentPlaceHolder_panel'})

        # 爬蟲
        dict_all = {}
        col_list = []
        for i in html.find_all('tr')[5].find_all('td'):
            dict_all.setdefault(i.text, {})
            col_list.append(i.text)
        for i, tr in enumerate(html.find_all('tr')[7:]):
            for j, td in enumerate(tr.find_all('td')):
                dict_all[col_list[j]][i] = td.text
        import pandas as pd
        print(pd.DataFrame(dict_all))
        
        # # 寫入資料庫
        # import pyodbc
        # server = '127.0.0.1, 1433' 
        # database = 'dbDemo' 
        # username = 'sa' 
        # password = 'as' 
        # cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + 
        #                       server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
        # cursor = cnxn.cursor()
        # # select
        # rows = cursor.fetchall()
        # for row in rows:
        #     print(row)
        #     print('---')
        # # insert
        # try:
        #     cnxn.autocommit = False
        #     params = [ ('A', 10, 5), ('B', 8, 6) ]
        #     cursor.executemany("insert into tProduct(fName, fPrice, fCost) values (?, ?, ?)", params)
        # except pyodbc.DatabaseError as err:
        #     cnxn.rollback()
        # else:
        #     cnxn.commit()
        # finally:
        #     cnxn.autocommit = True
        
        print("爬蟲成功，時間：" + today.strftime("%Y%m%d"))
    except:
        print("爬蟲失敗，時間：" + today.strftime("%Y%m%d"))

In [14]:
crawal()

開始爬蟲，時間：20200331
            市場            產品     上價     中價     下價 平均價(元/公斤) 跟前一交易日比較% 交易量(公斤)
0    104 台北二         11 椰子     16.0   13.5    8.1      12.9      +  2   1,944
1    104 台北二         22 棗子    152.2   92.3   54.9      96.8      +259   1,628
2    104 台北二         31 釋迦    119.1   93.9   79.6      96.1      + 75   1,965
3    104 台北二     32 釋迦 鳳梨釋迦   108.7   78.9   45.6      78.2      - 11     776
4    104 台北二          41 梅     78.8   46.6   23.1      48.3      +389   3,372
5    104 台北二         45 草莓     95.2   55.8   32.1      59.0      +  3   1,431
6    104 台北二      459 草莓 進口   658.3  344.5  291.6     396.7      - 32      40
7    104 台北二     51 百香果 改良種   139.0   97.5   57.1      97.7      + 24     913
8    104 台北二      70 小番茄 其他    76.9   50.2   26.3      50.8      + 34   2,163
9    104 台北二      72 小番茄 聖女    43.7   29.5   10.0      28.4      -  3   7,516
10   104 台北二      74 小番茄 玉女   183.2  116.9   55.6     117.9      - 17   4,192
11   104 台北二     812 火龍果 紅肉   199.7  146.1   77

# 定時爬蟲
    # 安裝：conda install -c conda-forge apscheduler

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
# BlockingScheduler
scheduler = BlockingScheduler()
    # 一般網站都是1:00點更新資料，所以每天凌晨一點啟動
scheduler.add_job(crawal, 'cron', day_of_week='0-6', hour=23, minute=21)
scheduler.start()

開始爬蟲，時間：20200331
            市場            產品     上價     中價     下價 平均價(元/公斤) 跟前一交易日比較% 交易量(公斤)
0    104 台北二         11 椰子     16.0   13.5    8.1      12.9      +  2   1,944
1    104 台北二         22 棗子    152.2   92.3   54.9      96.8      +259   1,628
2    104 台北二         31 釋迦    119.1   93.9   79.6      96.1      + 75   1,965
3    104 台北二     32 釋迦 鳳梨釋迦   108.7   78.9   45.6      78.2      - 11     776
4    104 台北二          41 梅     78.8   46.6   23.1      48.3      +389   3,372
5    104 台北二         45 草莓     95.2   55.8   32.1      59.0      +  3   1,431
6    104 台北二      459 草莓 進口   658.3  344.5  291.6     396.7      - 32      40
7    104 台北二     51 百香果 改良種   139.0   97.5   57.1      97.7      + 24     913
8    104 台北二      70 小番茄 其他    76.9   50.2   26.3      50.8      + 34   2,163
9    104 台北二      72 小番茄 聖女    43.7   29.5   10.0      28.4      -  3   7,516
10   104 台北二      74 小番茄 玉女   183.2  116.9   55.6     117.9      - 17   4,192
11   104 台北二     812 火龍果 紅肉   199.7  146.1   77